# neural network

### Import libraries

In [1]:
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'ce7454_training_pipeline.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    print(path_to_file)
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    os.chdir(path_to_file)
    !pwd

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import time
import copy

In [3]:
import torchvision
from torchvision import models
from torchvision import transforms
from torchvision import datasets
import pickle

In [4]:
!ls dataset 
!mkdir -p ./outputs

instagram_merged_expr_bin_1   test_part_utk_merged_expr_bin_1
instagram_merged_expr_bin_10  test_part_utk_merged_expr_bin_10
instagram_merged_expr_bin_3   test_part_utk_merged_expr_bin_3
instagram_merged_expr_bin_6   test_part_utk_merged_expr_bin_6
instagram_merged_raw	      test_part_utk_merged_raw
make_bin_dataset.sh	      us.zip
test_part_utk.tar.gz


In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, 
                 hidden_size3, hidden_size4, hidden_size5, 
                 hidden_size6, hidden_size7, hidden_size8,
                 hidden_size9, hidden_size10, hidden_size11, 
                 hidden_size12, hidden_size13, hidden_size14, 
                 hidden_size15, hidden_size16, hidden_size17, 
                 output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, hidden_size3),
            nn.ReLU(),
            nn.Linear(hidden_size3, hidden_size4),
            nn.ReLU(),
            nn.Linear(hidden_size4, hidden_size5),
            nn.ReLU(),
            nn.Linear(hidden_size5, hidden_size6),
            nn.ReLU(),
            nn.Linear(hidden_size6, hidden_size7),
            nn.ReLU(),
            nn.Linear(hidden_size7, hidden_size8),
            nn.ReLU(),
            nn.Linear(hidden_size8, hidden_size9),
            nn.ReLU(),
            nn.Linear(hidden_size9, hidden_size10),
            nn.ReLU(),
            nn.Linear(hidden_size10, hidden_size11),
            nn.ReLU(),
            nn.Linear(hidden_size11, hidden_size12),
            nn.ReLU(),
            nn.Linear(hidden_size12, hidden_size13),
            nn.ReLU(),
            nn.Linear(hidden_size13, hidden_size14),
            nn.ReLU(),
            nn.Linear(hidden_size14, hidden_size15),
            nn.ReLU(),
            nn.Linear(hidden_size15, hidden_size16),
            nn.ReLU(),
            nn.Linear(hidden_size16, hidden_size17),
            nn.ReLU(),
            nn.Linear(hidden_size17, output_size)
        )
        
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [6]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

### Gen the pickle

In [7]:
def gen_pickle(binsize, model_name):
    #
    # cpu or gpu
    #
    #device = torch.device("cuda")
    device = torch.device("cpu")
    print(device)

    #
    # dataset
    #
    expr_dataset = dict_data[binsize]
    img_pixels = (200, 200)
    channels = 3

    classes = dict_class[binsize]
    print("classes: ", classes)

    #
    # load the dataset
    #
    transform = transforms.Compose([
        # TODO: check this more later
        #transforms.Grayscale(num_output_channels=3),
        transforms.Resize(img_pixels),
        transforms.ToTensor()])
    img_data_expr = ImageFolderWithPaths(root=expr_dataset, transform=transform)
    data_loader_expr = torch.utils.data.DataLoader(img_data_expr, batch_size=1,shuffle=False)

    #
    # load the model parameters
    #
    if model_name == "MLP":
        net = dict_net[model_name](channels * img_pixels[0] * img_pixels[1], 512, 512,
                                   512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
                                   512, 512, 512, 512, 512, classes)
    else:
        net = dict_net[model_name](num_classes=classes)
    net.load_state_dict(torch.load(dict_model[binsize]))
    net.eval()

    #
    # get prediction results & parsing the img name
    #
    label_dict = dict(map(reversed, img_data_expr.class_to_idx.items()))
    print(label_dict)

    output = []

    for inputs, labels, paths in data_loader_expr:
        path, label, image = paths[0], labels.tolist()[0], inputs.tolist()[0]
        probs_tensor = net(inputs)
        probs = F.softmax(probs_tensor, dim=1)
        probs = probs.tolist()[0]

        #print('path:\t\t\t', path)
        #print('right label:\t\t', label_dict[label])
        #print(input.size())
        #print(image.size())
        #plt.imshow(trans(image))
        #print('predicted label:\t', label_dict[probs.index(max(probs))])
        #print('probs:\t\t', probs)

        output.append({"path": path,
                        "name": os.path.basename(path),
                        "label-str": label_dict[label], 
                        "pred-label-str": label_dict[probs.index(max(probs))],
                        "label-num": label, 
                        "pred-label-num": probs.index(max(probs)),
                        "probs":probs,})
        #break

    print(len(output))
    with open( dict_output[binsize], "wb" ) as f:
        pickle.dump(output, f)

    # load thing
    # with open( "save.p", "rb" ) as f:
    #    output = pickle.load( f )
    #

    # do bin 3 
    # do pickle for aoutput
    print("bin %d finished" % (binsize))

### Main

In [8]:

# ResNet or VGG or MLP
#model_name = "VGG"
#model_name = "ResNet"
#model_name = "MLP"

dict_class = {}
dict_model = {}
dict_data = {}
dict_output = {}
dict_net = {}

#for model_name in ["ResNet", "VGG", "MLP"]:
for model_name in ["MLP"]:
    for dataset in ["instagram", "test_part_utk"]:
    #for dataset in ["instagram"]:
        dict_class = {1:100, 
                      3:34, 
                      6:17, 
                      10:10,}
        dict_model = {1:"./saved_models/50_%s_merged_train_bin_1" % (model_name),
                      3:"./saved_models/50_%s_merged_train_bin_3" % (model_name),
                      6:"./saved_models/50_%s_merged_train_bin_6" % (model_name),
                      10:"./saved_models/50_%s_merged_train_bin_10" % (model_name),}
        dict_data = {1:"./dataset/%s_merged_expr_bin_1" % (dataset),
                     3:"./dataset/%s_merged_expr_bin_3" % (dataset),
                     6:"./dataset/%s_merged_expr_bin_6" % (dataset),
                     10:"./dataset/%s_merged_expr_bin_10" % (dataset),}
        dict_output = {1:"./outputs/pickle_%s_%s_bin_1" % (model_name, dataset),
                       3:"./outputs/pickle_%s_%s_bin_3" % (model_name, dataset),
                       6:"./outputs/pickle_%s_%s_bin_6" % (model_name, dataset),
                       10:"./outputs/pickle_%s_%s_bin_10" % (model_name, dataset),}
        dict_net = {"VGG" : models.vgg19_bn,
                    "ResNet" : models.resnet18,
                    "MLP" : MLP}
        gen_pickle(1, model_name)
        #gen_pickle(3, model_name)
        gen_pickle(6, model_name)
        gen_pickle(10, model_name)

cpu
classes:  100


/root/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 Quadro K2000 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


{0: 'age00to00', 1: 'age01to01', 2: 'age02to02', 3: 'age03to03', 4: 'age04to04', 5: 'age05to05', 6: 'age06to06', 7: 'age07to07', 8: 'age08to08', 9: 'age09to09', 10: 'age10to10', 11: 'age11to11', 12: 'age12to12', 13: 'age13to13', 14: 'age14to14', 15: 'age15to15', 16: 'age16to16', 17: 'age17to17', 18: 'age18to18', 19: 'age19to19', 20: 'age20to20', 21: 'age21to21', 22: 'age22to22', 23: 'age23to23', 24: 'age24to24', 25: 'age25to25', 26: 'age26to26', 27: 'age27to27', 28: 'age28to28', 29: 'age29to29', 30: 'age30to30', 31: 'age31to31', 32: 'age32to32', 33: 'age33to33', 34: 'age34to34', 35: 'age35to35', 36: 'age36to36', 37: 'age37to37', 38: 'age38to38', 39: 'age39to39', 40: 'age40to40', 41: 'age41to41', 42: 'age42to42', 43: 'age43to43', 44: 'age44to44', 45: 'age45to45', 46: 'age46to46', 47: 'age47to47', 48: 'age48to48', 49: 'age49to49', 50: 'age50to50', 51: 'age51to51', 52: 'age52to52', 53: 'age53to53', 54: 'age54to54', 55: 'age55to55', 56: 'age56to56', 57: 'age57to57', 58: 'age58to58', 59: 'a